In [4]:
import requests
import time

starttime = time.time()
print(requests.get('http://127.0.0.1:5555/bar').content)
print(requests.get('http://127.0.0.1:5555/foo').content)
print("消耗时间: ",time.time() -starttime)

b'<h1>bar!</h1>'
b'<h1>foo!</h1>'
消耗时间:  2.017179012298584


In [6]:
import selectors
import socket
import time
# from asynrequest import ParserHttp
class asynhttp:
    def __init__(self):
        self.selecter = selectors.DefaultSelector()

    def get(self,url,optiondict = None):
        global reqcount
        reqcount += 1
        s = socket.socket()
        s.setblocking(False)
        try:
            s.connect(('127.0.0.1',5555))
        except BlockingIOError:
            pass
        requset = 'GET %s HTTP/1.0\r\n\r\n' % url
        callback = lambda : self.send(s,requset)
        self.selecter.register(s.fileno(),selectors.EVENT_WRITE,callback)

    def send(self,s,requset):
        self.selecter.unregister(s.fileno())
        s.send(requset.encode())
        chunks = []
        callback = lambda: self.recv(s,chunks)
        self.selecter.register(s.fileno(),selectors.EVENT_READ,callback)

    def recv(self,s,chunks):
        self.selecter.unregister(s.fileno())
        chunk = s.recv(1024)
        if chunk:
            chunks.append(chunk)
            callback = lambda: self.recv(s,chunks)
            self.selecter.register(s.fileno(), selectors.EVENT_READ, callback)
        else:
            global reqcount
            reqcount -= 1
            # request_first,request_headers,request_content,_ = ParserHttp.parser(b''.join(chunks))
            # print("解析数据：",request_first,request_headers,request_content)
            print((b''.join(chunks)).decode())
            return (b''.join(chunks)).decode()

starttime = time.time()
reqcount = 0
asynhttper = asynhttp()
asynhttper.get('/bar')
asynhttper.get('/foo')
while reqcount:
    events = asynhttper.selecter.select()
    for event,mask in events:
        func = event.data
        func()
print("消耗时间：" ,time.time() - starttime)

HTTP/1.0 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 13
Server: Werkzeug/0.16.0 Python/3.7.4
Date: Thu, 20 May 2021 07:52:31 GMT

<h1>bar!</h1>
HTTP/1.0 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 13
Server: Werkzeug/0.16.0 Python/3.7.4
Date: Thu, 20 May 2021 07:52:31 GMT

<h1>foo!</h1>
消耗时间： 1.0070750713348389
